*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [42]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import VALID_METRICS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [277]:
# getting data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [321]:
# importing csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [336]:
# removing dataset users with less than 200 ratings and books with less than 100 ratings
num_user_ratings = df_ratings['user'].value_counts()
num_book_ratings = df_ratings['isbn'].value_counts()

new_df_ratings = df_ratings[df_ratings['user'].isin(num_user_ratings[num_user_ratings >= 200].index)]
new_df_ratings = new_df_ratings[new_df_ratings['isbn'].isin(num_book_ratings[num_book_ratings >= 100].index)]

In [337]:
# Merging the two datasets toether and droping duplicates 
# of each book and user

df = pd.merge(df_books,new_df_ratings,'inner',on = 'isbn')
df = df.drop_duplicates(['title','user'], keep = 'first')

In [339]:
# Creating a pivot table to isolate each book rating per row.
df = df.pivot(index='title', columns='user', values='rating')
df.fillna(0,inplace = True)


In [375]:
# add your code here - consider creating a new cell for each section of code
# since we are using only the titles to recommend the books, we only need the title column
titles = list(df.index)
len(titles)

673

In [387]:
# creating a feature variable that contains all reviews for each book per column
features = df.values

# converts a giving book title to feature rows
def book_to_features(title):
    index = titles.index(title)
    return features[index,:]

# gets the index of a feture and returns the book the index belongs to
def feature_index_to_book(index):
    return titles[index]

In [392]:
# function to return recommended books - this will be tested
def get_recommends(book):

    # Creating the knn model and fitting it to the values we have
    knn = NearestNeighbors(n_neighbors = 5,metric = 'cosine', algorithm = 'brute',p = 2)
    knn.fit(features)

    # passing in the book title we want to find similar books for
    row = book_to_features(book)
    distances, indexes = knn.kneighbors(np.reshape(row,[1,-1]),n_neighbors = 6,return_distance = True)
    distances = distances.tolist()[0]
    indexes = indexes.tolist()[0]

    recommended_books = []
    similar = []

    for distance, index in zip(distances, indexes):
        similar.append([feature_index_to_book(index),distance])
    similar = similar[::-1][:-1]
    recommended_books.append(book)
    recommended_books.append(similar)

    return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [390]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

# Function that checks if my code meets the requirement of the assignment
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864830970764]]]
You passed the challenge! 🎉🎉🎉🎉🎉
